In [24]:
import geopandas
import pandas as pd

In [46]:
communes_geo = geopandas.read_file("communes-20190101-shp")
type(communes_geo)

geopandas.geodataframe.GeoDataFrame

In [47]:
communes_geo.shape
communes = pd.read_csv('Communes.csv', sep =';', encoding='latin1', dtype=str) 
commune_bdv = pd.merge(communes_geo, communes, left_on = 'insee', right_on = 'CODGEO', how = 'inner').drop('CODGEO', axis = 1)
print(commune_bdv.shape)
commune_bdv.head()

(34970, 10)


,insee,nom,wikipedia,surf_ha,geometry,LIBGEO,BV2012,LIBBV2012,DEP,REG
0,97223,Saint-Esprit,fr:Saint-Esprit (Martinique),2318.0,"POLYGON ((-60.95617 14.55766, -60.95561 14.557...",Saint-Esprit,97222,Le Robert,972,02
1,97233,Le Morne-Vert,fr:Le Morne-Vert,1325.0,"POLYGON ((-61.15922 14.70645, -61.15917 14.706...",Le Morne-Vert,97209,Fort-de-France,972,02
2,97208,Fonds-Saint-Denis,fr:Fonds-Saint-Denis,2374.0,"POLYGON ((-61.16254 14.74252, -61.16232 14.743...",Fonds-Saint-Denis,97209,Fort-de-France,972,02
3,97224,Saint-Joseph,fr:Saint-Joseph (Martinique),4324.0,"POLYGON ((-61.08680 14.71891, -61.08678 14.719...",Saint-Joseph,97209,Fort-de-France,972,02
4,97212,Gros-Morne,fr:Gros-Morne,4601.0,"POLYGON ((-61.08483 14.72449, -61.08481 14.724...",Gros-Morne,97222,Le Robert,972,02


In [48]:
bassin_de_vie = commune_bdv.dissolve(by='BV2012')

In [50]:
bassin_de_vie.to_file('bassin_de_vie.shp')